In [374]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [398]:
learning_rate = 0.1
batch_size = 256
epochs = 300
g = 0.985

### read data

In [399]:
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')

In [400]:
np.unique(train['country'])

array([0, 1, 2])

In [401]:
train.head()

,row_id,country,store,product,num_sold,year,quarter,month,day,dayofweek,dayofmonth,dayofyear,weekofyear,weekday,is_weekend
0,0,0,0,1,329,2015,1,1,1,3,31,1,1,3,0
1,1,0,0,0,520,2015,1,1,1,3,31,1,1,3,0
2,2,0,0,2,146,2015,1,1,1,3,31,1,1,3,0
3,3,0,1,1,572,2015,1,1,1,3,31,1,1,3,0
4,4,0,1,0,911,2015,1,1,1,3,31,1,1,3,0


### data preprocessing

In [402]:
train = train.set_index('row_id')
test = test.set_index('row_id')


In [403]:
x_data = train.drop('num_sold', axis=1)
y_data = train.num_sold
std = StandardScaler()
x_data = std.fit_transform(x_data)
test = std.transform(test)

test = torch.tensor(test)
y_data = train[['num_sold']]
x_data = torch.tensor(x_data)
y_data = torch.tensor(y_data.values)

In [404]:
x_data = torch.tensor(x_data, dtype=torch.float32)
test = torch.tensor(test, dtype=torch.float32)

y_data = torch.tensor(y_data, dtype=torch.float32)

/tmp/ipykernel_997465/1023721312.py:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/tmp/ipykernel_997465/1023721312.py:2: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/tmp/ipykernel_997465/1023721312.py:4: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



### model train

In [405]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

In [406]:
dataset_train = TensorDataset(x_train, y_train)
loader_train = DataLoader(dataset_train, batch_size=256, shuffle=True)

dataset_test = TensorDataset(x_test, y_test)
loader_test = DataLoader(dataset_test, batch_size=256, shuffle=True)


In [407]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [408]:
import torch.nn.functional as F
n_feature = 13
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(n_feature, n_feature*2)
        self.fc2 = nn.Linear(n_feature*2, n_feature*4)
        self.bn1 = nn.BatchNorm1d(n_feature*4)
        self.fc3 = nn.Linear(n_feature*4, n_feature*8)
        self.fc4 = nn.Linear(n_feature*8, n_feature*16)
        self.bn2 = nn.BatchNorm1d(n_feature*16)
        # self.fc5 = nn.Linear(n_feature*16, n_feature*32)
        # self.fc6 = nn.Linear(n_feature*32, n_feature*64)
        # self.fc7 = nn.Linear(n_feature*64, n_feature*128)
        # self.bn3 = nn.BatchNorm1d(n_feature*128)
        # self.fc8 = nn.Linear(n_feature*128, n_feature*64)
        # self.fc9 = nn.Linear(n_feature*64, n_feature*32)
        # self.fc10 = nn.Linear(n_feature*32, n_feature*16)
        # self.bn4 = nn.BatchNorm1d(n_feature*16)
        # self.fc11 = nn.Linear(n_feature*16, n_feature*8)

        # self.bn5 = nn.BatchNorm1d(n_feature*8)
        # self.fc12 = nn.Linear(n_feature*8, n_feature*4)
        # self.fc13 = nn.Linear(n_feature*4, n_feature*2)
        # self.fc14 = nn.Linear(n_feature*2, n_feature)
        self.fc15 = nn.Linear(n_feature*16, 1)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.bn1(x))

        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.bn2(x))
        # x = F.relu(self.fc5(x)) 
        # x = F.relu(self.fc6(x) )
        # x = F.relu(self.fc7(x) )
        # x = self.bn3(x) 
        # x = F.relu(self.fc8(x) )
        # x = F.relu(self.fc9(x) )
        # x = F.relu(self.fc10(x) )
        # x = self.bn4(x)
        # x = F.relu(self.fc11(x) )
        # x = F.relu(self.bn5(x))
        # x = F.relu(self.fc12(x))
        # x = F.relu(self.fc13(x) )
        # x = F.relu(self.fc14(x))
        x = F.relu(self.fc15(x))
                
        return x

In [409]:
model = NeuralNetwork().to(device)
# model.apply(init_weights)
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=13, out_features=26, bias=True)
  (fc2): Linear(in_features=26, out_features=52, bias=True)
  (bn1): BatchNorm1d(52, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=52, out_features=104, bias=True)
  (fc4): Linear(in_features=104, out_features=208, bias=True)
  (bn2): BatchNorm1d(208, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc15): Linear(in_features=208, out_features=1, bias=True)
)


In [410]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        
        X = X.to(device)
        y = y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss = loss.item()
    loss = loss ** 0.5
    print(f"loss: {loss:>7f}")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    test_loss = test_loss ** 0.5
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [411]:
from math import exp
import math

In [412]:
loss_fn = nn.SmoothL1Loss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=g)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(loader_train, model, loss_fn, optimizer)
    test_loop(loader_test, model, loss_fn)
    scheduler.step()
    # print(optimizer.param_groups[0]["lr"])
print("Done!")

Epoch 1
-------------------------------
loss: 9.027496
Test Error: 
 Avg loss: 6.986893 

Epoch 2
-------------------------------
loss: 10.940895
Test Error: 
 Avg loss: 6.666197 

Epoch 3
-------------------------------
loss: 11.631951
Test Error: 
 Avg loss: 6.328080 

Epoch 4
-------------------------------
loss: 8.245991
Test Error: 
 Avg loss: 6.218343 

Epoch 5
-------------------------------
loss: 8.620891
Test Error: 
 Avg loss: 6.144149 

Epoch 6
-------------------------------
loss: 8.372263
Test Error: 
 Avg loss: 6.681160 

Epoch 7
-------------------------------
loss: 9.344016
Test Error: 
 Avg loss: 5.922385 

Epoch 8
-------------------------------
loss: 5.986565
Test Error: 
 Avg loss: 5.865845 

Epoch 9
-------------------------------
loss: 9.622318
Test Error: 
 Avg loss: 5.823307 

Epoch 10
-------------------------------
loss: 10.764380
Test Error: 
 Avg loss: 6.246286 

Epoch 11
-------------------------------
loss: 10.324581
Test Error: 
 Avg loss: 6.191246 

Epoc

KeyboardInterrupt: 

In [390]:
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)


In [391]:
x_train = x_train.to(device)
x_test = x_test.to(device)

train_pred = model(x_train)
test_pred = model(x_test)

train_score = SMAPE(y_train.cpu().detach().numpy(), train_pred.cpu().detach().numpy())
test_score = SMAPE(y_test.cpu().detach().numpy(), test_pred.cpu().detach().numpy())

print('train_score ', train_score, 'test_score', test_score)

train_score  4.929468 test_score 5.304886


In [392]:
test = test.to(device)

In [393]:
test_pred = model(test)

In [394]:
train_pred.cpu()

tensor([[511.4787],
        [352.5506],
        [296.9967],
        ...,
        [310.5991],
        [198.9089],
        [ 86.7982]], grad_fn=<ReluBackward0>)

In [395]:
unique, counts = np.unique(y_train.cpu().detach().numpy(), return_counts=True)
unique_p, counts_p = np.unique(np.round(train_pred.cpu().detach().numpy()), return_counts=True)
d_p = pd.DataFrame(index = unique_p,data = counts_p,columns=['count_pred'])
d = pd.DataFrame(index= unique,data=  counts,columns=['count_true'])
d1 = d.join(d_p)
d1 = d1.fillna(0)

In [396]:
import plotly.express as px

fig = px.line(d1, x= d.index, y = ['count_true', 'count_pred'])
fig.show()

In [397]:
submission = pd.read_csv('sample_submission.csv')
submission.num_sold = test_pred.cpu().detach().numpy().reshape(-1)
submission.to_csv('submission.csv', index=False)